# DNBLab Jupyter Notebook Tutorial

## Digitalisierte Inhaltsverzeichnisse: Datenabfrage, Auslieferung und Volltextanalyse

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage zu digitalisierten Inhaltsverzeichnissen über die SRU-Schnittstelle und umfasst das temporäre Speichern der Inhaltsverzeichnisse als Textdateien sowie die Analyse der Volltexte nach 

* [1. Häufigkeit eines beliebigen Suchwortes](#Teil1)
* [2. Vorkommen des Suchwortes in den einzelnen Textdateien](#Teil2)
* [3. Top 20 häufigste Wörter in den Textdateien](#Teil3)


## Einrichten der Arbeitsumgebung

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden.

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd
from collections import Counter

## SRU-Abfrage mit Ausgabe in MARC21-xml

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von Records aus. Bei mehr als 100 Records werden weitere Datensätze mit "&startRecord=101" abgerufen (mögliche Werte 1 bis 99.000). Weitere Informationen und Funktionen der SRU- Schnittstelle werden unter https://www.dnb.de/sru beschrieben.

In [2]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Durchsuchen eines MARC-Feldes

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen heraus und liefert diese als Dictionary zurück. Die Schlüssel-Werte-Paare können beliebig agepasst und erweitert werden. In diesem Fall werden nur die Permalinks zu den digitalisierten Inhaltsverzeichnissen als "link" ausgegeben.

In [3]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #umfang = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
         # Archivlink
    link = xml.xpath("marc:datafield[@ind1 = ' ']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
        #link = unicodedata.normalize("NFC", urn)
    except:
        link = 'fail'
        
         # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = 'fail'
        
    meta_dict = {"idn":idn,
                 "titel":titel,
                 "link":link,
                 "umfang":umfang
                               }
    
    return meta_dict

Über verschiedenen Indices https://services.dnb.de/sru/dnb?operation=explain&version=1.1 kann die SRU-Abfrage "dnb_sru" mittels CQL https://www.dnb.de/DE/Service/Hilfe/Katalog/kataloghilfeExpertensuche.html eingeschränkt werden. Im Folgenden Code wird die Abfrage über das Stichwort "Sandwespe" im Volltextindex der digitalisierten Inhaltsverzeichnisse eingeschränkt. Durch Anpassen der SRU-Abfrage kann die Trefferliste beliebig geändert werden.

In [9]:
records = dnb_sru('tit=Faust and vlg=Ungar and cod=d0* and location=onlinefree')
print(len(records), 'Ergebnisse')

2 Ergebnisse


## Beispielanzeige zur weiteren Bearbeitung

Mit der Bibliothek Pandas für Python wird das Ergebnis (Dictionary-Element "link") als Dataframe ausgegeben.

In [10]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

#Die Ausgabe der ermittelten Links kann je nach Bedarf über verschiedene Funktionen erfolgen:
#print(df.to_string(index=False))
#HTML(df.to_html(index=False))
#document = df.to_dict(orient='list')
#print(document)

,idn,titel,link,umfang
0,1032408820,Faust,https://d-nb.info/1032408820/34,138 S.
1,1032408839,Faust,https://d-nb.info/1032408839/34,221 S.


## Download der Textdateien 

Mit der folgenden Funktion df.to_csv() werden die Ergebnisse als "links.csv" in das Jupyter-Verzeichnins der Einstiegsseite abgelegt und können dort heruntergeladen werden. 

In [15]:
!wget https://d-nb.info/1032408839/34

--2021-09-06 16:28:17--  https://d-nb.info/1032408839/34
Resolving d-nb.info (d-nb.info)... 193.175.100.223
Connecting to d-nb.info (d-nb.info)|193.175.100.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘34’

34                      [       <=>          ] 996.73M  13.1MB/s    in 68s     

2021-09-06 16:29:25 (14.7 MB/s) - ‘34’ saved [1045152068]



Mit wget werden alle in der CSV-Datei gespeicherten Links heruntergeladen und als Textdateien (text, text.1, text.2, usw.) im temporären Jupyter-Verzeichnis gespeichert (Downloadlimit 1MB). 

In [16]:
!unzip 34

Archive:  34
  inflating: 1032408839_00000220.tif  
  inflating: 1032408839_00001470.tif  
  inflating: 1032408839_00001760.tif  
  inflating: 1032408839_00000880.tif  
  inflating: 1032408839_00000360.tif  
  inflating: 1032408839_00001900.tif  
  inflating: 1032408839_00001100.tif  
  inflating: 1032408839_00000170.tif  
  inflating: 1032408839_00000910.tif  
  inflating: 1032408839_00001600.tif  
  inflating: 1032408839_00001530.tif  
  inflating: 1032408839_00000100.tif  
  inflating: 1032408839_00000180.tif  
  inflating: 1032408839_00000770.tif  
  inflating: 1032408839_00000230.tif  
  inflating: 1032408839_00001930.tif  
  inflating: 1032408839_00002250.tif  
  inflating: 1032408839_00000890.tif  
  inflating: 1032408839_00001560.tif  
  inflating: 1032408839_00000060.tif  
  inflating: 1032408839_00000050.tif  
  inflating: 1032408839_00000740.tif  
  inflating: 1032408839_00002320.tif  
  inflating: 1032408839_00001110.tif  
  inflating: 1032408839_00001780.tif  
  inflating:

  inflating: 1032408839_00000790.tif  
  inflating: 1032408839_00001650.tif  
  inflating: 1032408839_00000120.tif  
  inflating: 1032408839_00001580.tif  
  inflating: 1032408839_00001020.tif  
  inflating: 1032408839_00000510.tif  
  inflating: 1032408839_00001770.tif  
  inflating: 1032408839_00000150.tif  
  inflating: 1032408839_00000430.tif  
  inflating: 1032408839_00000600.tif  
  inflating: 1032408839_00002080.tif  
  inflating: 1032408839_00001420.tif  
  inflating: 1032408839_00001050.tif  
  inflating: 1032408839_00002000.tif  
  inflating: 1032408839_00000710.tif  
  inflating: 1032408839_00002290.tif  
  inflating: 1032408839_00000410.tif  
  inflating: 1032408839_00000470.tif  
  inflating: 1032408839_00001660.tif  
  inflating: 1032408839_00002160.tif  
  inflating: 1032408839_00002090.tif  
  inflating: 1032408839_00000950.tif  
  inflating: 1032408839_00001590.tif  
  inflating: 1032408839_00002050.tif  
  inflating: 1032408839_00001690.tif  
  inflating: 1032408839_0

## Volltextanalyse 

### 1. Häufigkeit eines beliebigen Suchwortes<a class="anchor" id="Teil1"></a>

Die heruntergeladenen Textdateien können nach einem Suchwort, z.B. search = "biene" durchsucht werden. Hierbei wird die Groß- und Kleinschreibung beachtet.
Als Antwort werden die durchsuchten Textdateien genannt und die Häufigkeit des gesuchten Wortes in den genannten Dateien ausgegeben. Dabei entspricht die Dateibenennung den im Verzeichnis heruntergeladen Textdateien (text, text1, text2 usw.).
Das Suchwort kann beliebig geändert und die Suche durch Ausführen des Codes angepasst werden.

In [19]:
# Eingabe des Suchwortes mit Button zur Übernahme der eingegebenen Werte 

from IPython.display import display 
import ipywidgets as widgets 
from ipywidgets import interact, Layout 

tb1 = widgets.Text(value = 'Tag', description ='Suchwort: ');display(tb1);


button = widgets.Button(description='Übernehmen!', layout=Layout(width='200px')); 
button.style.button_color='lightgreen';display(button); 
def on_button_clicked(sender): 
    a = tb1.value; 
    print('Folgendes Suchwort wurde übernommen: ' + str(tb1.value))     
button.on_click(on_button_clicked)

Text(value='Tag', description='Suchwort: ')

Button(description='Übernehmen!', layout=Layout(width='200px'), style=ButtonStyle(button_color='lightgreen'))

In [18]:
filename = '1032408839_OCR.txt'
search = str(tb1.value)
with open(filename) as f:
    for num, line in enumerate(f, 1):
        if search in line:
            print('%s - found at line:' % search, num)

Tag - found at line: 58
Tag - found at line: 72
Tag - found at line: 84
Tag - found at line: 100
Tag - found at line: 214
Tag - found at line: 263
Tag - found at line: 306
Tag - found at line: 307
Tag - found at line: 350
Tag - found at line: 500
Tag - found at line: 503
Tag - found at line: 625
Tag - found at line: 824
Tag - found at line: 840
Tag - found at line: 878
Tag - found at line: 959
Tag - found at line: 1414
Tag - found at line: 1468
Tag - found at line: 1515
Tag - found at line: 1612
Tag - found at line: 1636
Tag - found at line: 1777
Tag - found at line: 2082
Tag - found at line: 2355
Tag - found at line: 2512
Tag - found at line: 2604
Tag - found at line: 2628
Tag - found at line: 2794
Tag - found at line: 2992
Tag - found at line: 3172
Tag - found at line: 3188
Tag - found at line: 3718
Tag - found at line: 3877
Tag - found at line: 3968
Tag - found at line: 4211
Tag - found at line: 4218
Tag - found at line: 4298
Tag - found at line: 4912
Tag - found at line: 4960
Tag -